In [1]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 40.7834,
	"longitude": -73.9663,
	"start_date": "2022-01-01",
	"end_date": "2025-02-01",
	"hourly": ["temperature_2m", "precipitation", "rain", "snowfall", "snow_depth", "wind_speed_10m"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(1).ValuesAsNumpy()
hourly_rain = hourly.Variables(2).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(3).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(4).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(5).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

Coordinates 40.808433532714844°N -74.0198974609375°E
Elevation 45.0 m asl
Timezone NoneNone
Timezone difference to GMT+0 0 s
                           date  temperature_2m  precipitation  rain  \
0     2022-01-01 00:00:00+00:00          8.3005            0.0   0.0   
1     2022-01-01 01:00:00+00:00          9.0005            0.0   0.0   
2     2022-01-01 02:00:00+00:00          8.2505            0.0   0.0   
3     2022-01-01 03:00:00+00:00          7.9005            0.0   0.0   
4     2022-01-01 04:00:00+00:00          7.7505            0.0   0.0   
...                         ...             ...            ...   ...   
27067 2025-02-01 19:00:00+00:00         -0.2495            0.0   0.0   
27068 2025-02-01 20:00:00+00:00         -0.6495            0.0   0.0   
27069 2025-02-01 21:00:00+00:00         -1.5995            0.0   0.0   
27070 2025-02-01 22:00:00+00:00         -3.0995            0.0   0.0   
27071 2025-02-01 23:00:00+00:00         -4.4995            0.0   0.0   

       sno

In [3]:
import datetime as dt

In [5]:
hourly_dataframe["date"] = hourly_dataframe["date"].apply(lambda x: x.tz_convert(None))

In [13]:
hourly_dataframe.to_csv("data/weather_test.csv")